### KoBERT기반 컬럼예측 CTA를 위한 코드입니다

In [1]:
!pip install transformers
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 파인튜닝용 training 코드

In [15]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from transformers import BertTokenizer, BertModel, get_scheduler

# ✅ 1. 데이터 불러오기
df = pd.read_csv("train_mixed_name.csv", encoding='utf-8-sig')
label_map = {label: i for i, label in enumerate(df["label"].unique())}
df["label_id"] = df["label"].map(label_map)

# ✅ 2. 토크나이저 및 BERT 모델 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
bertmodel = BertModel.from_pretrained('monologg/kobert')

# ✅ 3. 커스텀 Dataset 정의
class NameDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoded = tokenizer(self.texts[idx], padding='max_length', truncation=True, max_length=32, return_tensors="pt")
        return {
            'input_ids': encoded['input_ids'].squeeze(),
            'attention_mask': encoded['attention_mask'].squeeze(),
            'label': torch.tensor(self.labels[idx])
        }

dataset = NameDataset(df["text"].tolist(), df["label_id"].tolist())
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# ✅ 4. 분류기 모델 정의
class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size=768, num_classes=4):
        super().__init__()
        self.bert = bert
        self.classifier = nn.Linear(hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # [CLS] 토큰 임베딩
        return self.classifier(pooled_output)

# ✅ 5. 학습 준비
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bertmodel, num_classes=len(label_map)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
loss_fn = nn.CrossEntropyLoss()

# ✅ 6. Scheduler 설정
num_epochs = 7
num_training_steps = len(dataloader) * num_epochs
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# ✅ 7. 학습 루프
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"✅ Epoch {epoch+1} 완료 - 평균 Loss: {avg_loss:.4f}")

# ✅ 8. 모델 저장
torch.save(model.state_dict(), "kobert_name_finetuned.pt")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


✅ Epoch 1 완료 - 평균 Loss: 0.8601
✅ Epoch 2 완료 - 평균 Loss: 0.5106
✅ Epoch 3 완료 - 평균 Loss: 0.3266
✅ Epoch 4 완료 - 평균 Loss: 0.3110
✅ Epoch 5 완료 - 평균 Loss: 0.3059
✅ Epoch 6 완료 - 평균 Loss: 0.3646
✅ Epoch 7 완료 - 평균 Loss: 0.3579


### 컬럼 예측

In [16]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from torch import nn
import torch.nn.functional as F

# ✅ 훈련 시 사용한 라벨 맵 고정
label_map = {"성명": 0, "출생": 1, "성별": 2, "지역": 3}

# ✅ 모델 정의
class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size=768, num_classes=4):
        super().__init__()
        self.bert = bert
        self.classifier = nn.Linear(hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        return self.classifier(pooled_output)

# ✅ 모델 및 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")
bertmodel = BertModel.from_pretrained("monologg/kobert")
model = BERTClassifier(bertmodel, num_classes=4)
model.load_state_dict(torch.load("kobert_name_finetuned.pt", map_location="cpu"))
model.eval()

# ✅ 성명 예측 함수
def predict_is_name(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=32)
    with torch.no_grad():
        logits = model(inputs['input_ids'], inputs['attention_mask'])
        probs = torch.softmax(logits, dim=1)
        return probs[0][label_map["성명"]].item()

# ✅ 판결문 CSV 불러오기
df = pd.read_csv("judgement.csv", encoding='cp949')

# ✅ 컬럼별 평균 성명확률 계산
results = []
for col in df.columns:
    values = df[col].dropna().astype(str).tolist()[:100]
    probs = [predict_is_name(val) for val in values]
    avg_prob = sum(probs) / len(probs) if probs else 0
    results.append((col, avg_prob))

# ✅ 출력
results.sort(key=lambda x: x[1], reverse=True)
print("🧠 CSV에서 '성명' 컬럼으로 가장 유력한 후보:")
for col, score in results:
    print(f" {col}: 성명일 확률 평균 {score:.4f}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


🧠 CSV에서 '성명' 컬럼으로 가장 유력한 후보:
 이름_별명: 성명일 확률 평균 0.9995
 주문: 성명일 확률 평균 0.1504
 생산년도: 성명일 확률 평균 0.0008
 마이크로필름번호: 성명일 확률 평균 0.0008
 판결날짜: 성명일 확률 평균 0.0008
 연번: 성명일 확률 평균 0.0008
 본적주소: 성명일 확률 평균 0.0008
 판결문_번역본_제공: 성명일 확률 평균 0.0008
 판결문_원문_제공: 성명일 확률 평균 0.0008
 관리번호: 성명일 확률 평균 0.0008
 당시나이: 성명일 확률 평균 0.0008
 사건개요: 성명일 확률 평균 0.0008
 죄명: 성명일 확률 평균 0.0008


In [23]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from torch import nn
import torch.nn.functional as F

# ✅ 훈련 시 사용한 라벨 맵 고정
label_map = {"성명": 0, "출생": 1, "성별": 2, "주소": 3}

# ✅ 모델 정의
class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size=768, num_classes=4):
        super().__init__()
        self.bert = bert
        self.classifier = nn.Linear(hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        return self.classifier(pooled_output)

# ✅ 모델 및 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")
bertmodel = BertModel.from_pretrained("monologg/kobert")
model = BERTClassifier(bertmodel, num_classes=4)
model.load_state_dict(torch.load("kobert_name_finetuned.pt", map_location="cpu"))
model.eval()

# ✅ 성명 예측 함수
def predict_is_name(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=32)
    with torch.no_grad():
        logits = model(inputs['input_ids'], inputs['attention_mask'])
        probs = torch.softmax(logits, dim=1)
        return probs[0][label_map["성명"]].item()

# ✅ 판결문 CSV 불러오기
df = pd.read_csv("judgement2.csv", encoding='cp949')

# ✅ 컬럼별 평균 성명확률 계산
results = []
for col in df.columns:
    values = df[col].dropna().astype(str).tolist()[:100]
    probs = [predict_is_name(val) for val in values]
    avg_prob = sum(probs) / len(probs) if probs else 0
    results.append((col, avg_prob))

# ✅ 출력
results.sort(key=lambda x: x[1], reverse=True)
print("🧠 CSV에서 '성명' 컬럼으로 가장 유력한 후보:")
for col, score in results:
    print(f" {col}: 성명일 확률 평균 {score:.4f}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


🧠 CSV에서 '성명' 컬럼으로 가장 유력한 후보:
 이름_이름: 성명일 확률 평균 0.9995
 주문: 성명일 확률 평균 0.1504
 생산년도: 성명일 확률 평균 0.0008
 마이크로필름번호: 성명일 확률 평균 0.0008
 판결날짜: 성명일 확률 평균 0.0008
 연번: 성명일 확률 평균 0.0008
 본적주소: 성명일 확률 평균 0.0008
 판결문_번역본_제공: 성명일 확률 평균 0.0008
 판결문_원문_제공: 성명일 확률 평균 0.0008
 관리번호: 성명일 확률 평균 0.0008
 당시나이: 성명일 확률 평균 0.0008
 사건개요: 성명일 확률 평균 0.0008
 죄명: 성명일 확률 평균 0.0008
